Train 데이터를 Paragraph 단위로 자른 후 Knowledge base 가 큰 LLM 으로 relabeling 

In [1]:
import pandas as pd
import numpy as np
from collections import Counter
import re
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoModelForSequenceClassification
from tqdm import tqdm
import random

In [2]:
data_base = "../data"

In [3]:
train_df = pd.read_csv(f"{data_base}/train.csv")
train_df.head()

,title,full_text,generated
0,카호올라웨섬,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0
1,청색거성,"천문학에서 청색거성(靑色巨星, )은 광도 분류에서 III형(거성) 또는 II형(밝은...",0
2,엘자스-로트링겐 평의회 공화국,엘자스-로트링겐 평의회 공화국은 1차대전 말기 독일 혁명 와중에 엘자스-로트링겐에서...,0
3,윌리엄 페니 브룩스,"윌리엄 페니 브룩스(, 1809년 8월 13일 ~ 1895년 12월 11일)는 잉글...",0
4,미그로,"미그로 또는 미그로스(""Migros"")는 스위스 최대 소매 회사이자, 최대 슈퍼마켓...",0


In [4]:
checkpoint_path = f"../labeling_ckpt/checkpoint-200000"  # 원하는 checkpoint
tokenizer = AutoTokenizer.from_pretrained(checkpoint_path)
model = AutoModelForSequenceClassification.from_pretrained(checkpoint_path)
model.to("cuda" if torch.cuda.is_available() else "cpu")
model.eval()

2025-07-18 08:47:33.164393: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2025-07-18 08:47:33.172099: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752796053.180439 3261954 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752796053.182738 3261954 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-07-18 08:47:33.191727: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instr

RobertaForSequenceClassification(
  (roberta): RobertaModel(
    (embeddings): RobertaEmbeddings(
      (word_embeddings): Embedding(32000, 1024, padding_idx=1)
      (position_embeddings): Embedding(514, 1024, padding_idx=1)
      (token_type_embeddings): Embedding(1, 1024)
      (LayerNorm): LayerNorm((1024,), eps=1e-05, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): RobertaEncoder(
      (layer): ModuleList(
        (0-23): 24 x RobertaLayer(
          (attention): RobertaAttention(
            (self): RobertaSdpaSelfAttention(
              (query): lora.Linear(
                (base_layer): Linear(in_features=1024, out_features=1024, bias=True)
                (lora_dropout): ModuleDict(
                  (default): Dropout(p=0.1, inplace=False)
                )
                (lora_A): ModuleDict(
                  (default): Linear(in_features=1024, out_features=8, bias=False)
                )
                (lora_B): ModuleDict(

In [5]:
def split_sentences(text):
    return [s.strip() for s in text.split('\n') if s.strip()]


def response_postprocessing(decoded):
    # 공백 제거
    decoded = decoded.strip()

    # 뒤쪽 100자만 잘라서 보면 속도도 빠르고 의미도 보장됨
    tail = decoded[-100:].upper()  # 대소문자 구분 없게

    if re.search(r'\bAI\b', tail[::-1]):
        return 1
    elif re.search(r'\bHUMAN\b', tail[::-1]):
        return 0
    else:
        return 0.5

In [6]:
def make(sentence, tokenizer, model):
    valid_keys = {"input_ids", "attention_mask"}

    inputs = tokenizer(
            sentence,
            truncation=True,
            padding='max_length',
            max_length=512,
            stride=256,  # ✅ 겹치게 자름
            return_overflowing_tokens=True,
            return_tensors="pt"
        )
    
    # segment 중 하나 랜덤 선택
    n_segments = inputs["input_ids"].size(0)
    seg_idx = random.randint(0, n_segments - 1)

    item = {
        k: v[seg_idx] for k, v in inputs.items() if k != "overflow_to_sample_mapping"
    }

    inputs = {k: v.unsqueeze(0).to(model.device) for k, v in item.items() if k in valid_keys}

    # Longformer는 token_type_ids 없음
    inputs.pop("token_type_ids", None)

    outputs = model(**inputs)
    logits = outputs.logits
    probs = torch.softmax(logits, dim=-1)
    preds = torch.argmax(probs, dim=-1)

    return preds

In [7]:
output_csv = f"{data_base}/pseudo_labeling.csv"
# 초기 딕셔너리
parsing_df = {
    "title": [],
    "paragraph_idx": [],
    "paragraph": [],
    "paragraph_label": [],
    "document_label": []
}

for _, row in tqdm(train_df.iterrows(), desc="processing", total=len(train_df)):
    title = row["title"]
    text = row["full_text"]
    document_label = row["generated"]

    split_text = split_sentences(text)

    for idx, paragraph in enumerate(split_text):
        if document_label == 0:
            paragraph_label = 0
        else:
            paragraph_label = make(paragraph, tokenizer, model).cpu().item()

        # 딕셔너리에 추가
        parsing_df["title"].append(title)
        parsing_df["paragraph_idx"].append(idx)
        parsing_df["paragraph"].append(paragraph)
        parsing_df["paragraph_label"].append(paragraph_label)
        parsing_df["document_label"].append(document_label)

# 최종 DataFrame 생성
parsing_df = pd.DataFrame(parsing_df)
parsing_df.to_csv(f"{output_csv}", index=False)

processing:   0%|          | 0/97172 [00:00<?, ?it/s]

processing: 100%|██████████| 97172/97172 [44:50<00:00, 36.12it/s]  


In [ ]:
train_csv = pd.read_csv(output_csv)
train_csv = train_csv.rename(columns={
    'paragraph': 'full_text'
    })

train_csv.head()

,Unnamed: 0,title,paragraph_idx,full_text,paragraph_label,document_label
0,0,카호올라웨섬,0,카호올라웨섬은 하와이 제도를 구성하는 8개의 화산섬 가운데 하나로 면적은 115.5...,0,0
1,1,카호올라웨섬,1,마우이섬에서 남서쪽으로 약 11km 정도 떨어진 곳에 위치하며 라나이섬의 남동쪽에 ...,0,0
2,2,카호올라웨섬,2,1000년경부터 사람이 거주했으며 해안 지대에는 소규모 임시 어촌이 형성되었다. 섬...,0,0
3,3,카호올라웨섬,3,1830년대에는 하와이 왕국의 카메하메하 3세 국왕에 의해 남자 죄수들의 유형지로 ...,0,0
4,4,카호올라웨섬,4,1910년부터 1918년까지 하와이 준주가 섬의 원래 모습을 복원하기 위해 이 섬을...,0,0


In [9]:
print(f"데이터 개수 : {len(train_csv)}")
avg_text = sum([len(i) for i in train_csv["full_text"]])/len(train_csv)
print(f"평균 text 길이 : {avg_text}")
max_length = max([len(i) for i in train_csv["full_text"]])
min_length = min([len(i) for i in train_csv["full_text"]])
print(f"가장 길이가 긴거 : {max_length}")
print(f"가장 길이가 짧은거 : {min_length}")

데이터 개수 : 1226364
평균 text 길이 : 181.50835804051653
가장 길이가 긴거 : 19124
가장 길이가 짧은거 : 9


In [10]:
print(f"클래스 비율 : {sum(train_csv['paragraph_label'])/len(train_csv)}")
print(f"가중치 : {len(train_csv)/sum(train_csv['paragraph_label'])}")

클래스 비율 : 0.03529294728155751
가중치 : 28.33427290790629


In [ ]:
filter_df = train_csv[train_csv['document_label'] == train_csv['paragraph_label']]

list_1_1 = []
for _,i in filter_df.iterrows():
    if i['paragraph_label'] == 1:
        list_1_1.append(i)

print(f"1 == 1 : {len(list_1_1)}")
print(f"0 == 1 : {len(train_csv[train_csv['document_label'] != train_csv['paragraph_label']])}")

1 == 1 : 43282
0 == 1 : 57430


In [ ]:
# 1. 먼저 generated == 1 인 row만 필터링
gen1_df = train_csv[train_csv["document_label"] == 1]

# 2. 그 중 title 별로 paragraph_label 평균 계산
grouped = gen1_df.groupby("title")["paragraph_label"].mean()

# 3. 평균이 정확히 0인 title만 선택
zero_titles = grouped[grouped == 0].index.tolist()

print(f"paragraph_label 평균이 0인 title 개수: {len(zero_titles)}")
print("예시:", zero_titles[:10])

# 전체 title 수 (중복 제거)
total_titles = train_csv["title"].nunique()

# generated == 1 인 title만 필터링
generated_1_titles = train_csv[train_csv["document_label"] == 1]["title"].unique()

# 개수와 비율 계산
num_generated_1 = len(zero_titles)
ratio = num_generated_1 / total_titles

print(f"전체 title 수: {total_titles}")
print(f"generated == 1 인 title 수: {num_generated_1}")
print(f"비율: {ratio:.4f} ({ratio*100:.2f}%)")


paragraph_label 평균이 0인 title 개수: 185
예시: ['1967–1970', '2013년 AFC 챔피언스리그', '2017년 일본 시리즈', '2021년 아프리카 네이션스컵', 'Around the Fur', 'B사감과 러브레터', 'C3 식물', 'E.T.', 'IC 1101', 'IPTG']
전체 title 수: 97172
generated == 1 인 title 수: 185
비율: 0.0019 (0.19%)


In [ ]:
filtered_df = train_csv[~train_csv["title"].isin(zero_titles)]
filtered_df.to_csv(output_csv, index=False)
